# First model

## Setup

#### Imports

In [153]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd

from amftrack.util.sys import storage_path
from amftrack.util.file import chose_file

In [47]:
tf.config.functions_run_eagerly()

False

In [133]:
test_file = chose_file(section_path)

In [5]:
dataset_path = os.path.join(storage_path, "width3", "dataset_test")

#### Utils

In [151]:
def get_nodes(ch):
    nodes = ch.split("-")
    print(nodes)
    return nodes[0], nodes[1]

## I/ Data pipeline

Some remarks:

- We use different edges for training validation and test

### 1/ Section profile

In [132]:
section_path = os.path.join(dataset_path, "Img")

In [131]:
@tf.function
def load_image(filename):
    "Handle one edge"
    # 1/ Slices from the edge
    raw = tf.io.read_file(filename) # open the file
    image = tf.image.decode_png(raw, channels=1, dtype = tf.uint8)
    print("Initial shape: ", image.shape)
    image = tf.squeeze(image) # removing the last axis
    # image.set_shape([None, 120, 1]) # TODO: chose here only part of the array
    print("Final shape: ", image.shape)
    
    # 2/ Labels
    # TODO
    return tf.data.Dataset.from_tensor_slices(image)

In [142]:
def reader_dataset(directory_path, repeat=1, n_readers=5, shuffle_buffer_size = 1000, batch_size=32):
    """
    Take as input the directory containing the data points.
    And return a tf.Dataset object iterating through the batches.
    """
    filepaths = [os.path.join(directory_path, file) for file in os.listdir(directory_path)]
    path_dataset = tf.data.Dataset.list_files(filepaths) # yield file names randomly
    # TODO: make buffer size to size of the dataset
    # TODO: get a dataset object from one file
    general_dataset = path_dataset.interleave(lambda filepath: load_image(filepath), cycle_length=n_readers)
    general_dataset = general_dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return general_dataset.batch(batch_size).prefetch(1)

In [166]:
data = pd.read_csv(os.path.join(dataset_path, "data.csv")) #.set_index('edge')
#del data['Unnamed: 0']

In [170]:
data[data["edge"] == f"(Node({482}),Node({478}))"]

,Unnamed: 0,edge,width


In [172]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  43 non-null     int64  
 1   edge        43 non-null     object 
 2   width       43 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ KB


### 2/ Coordinates

## II/ First model

# Brouillon

In [ ]:
image_dir = 'hypothesis/temp'
image_root = pathlib.Path(image_dir)
list_ds = tf.data.Dataset.list_files(str(image_root/'*.png'))
for f in list_ds:
  image = tf.io.read_file(f)
  image = tf.io.decode_png(image)

In [ ]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

labeled_ds = list_ds.map(process_path)

for image_raw, label_text in labeled_ds.take(1):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())


In [30]:
@tf.function
def load_image(filename):
    raw = tf.io.read_file(filename)
    image = tf.image.decode_png(raw, channels=1)
    # the `print` executes during tracing.
    print("Initial shape: ", image.shape)
    image.set_shape([28, 28, 3])
    print("Final shape: ", image.shape)
    return image


In [ ]:
def process_path(file_path):
    label = tf.strings.split(file_path, os.sep)[-2]

In [ ]:
def to_dataset(dataframe, shuffle=True, batch_size=32):
    labels = dataframe.pop('label')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [ ]:
import tensorflow as tf
filenames = ["./img1.jpg", "./img2.jpg"]
labels = ["A", "B"]

def load_image(filePath, label):
    print('Loading File: {}' + filePath)
    raw_bytes = tf.io.read_file(filePath)
    image = tf.io.decode_image(raw_bytes, expand_animations = False)
    return tf.data.Dataset.from_tensors((image, label))

dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
dataset = dataset.interleave(lambda x, y: load_image(x, y), cycle_length=4)

for i in dataset.as_numpy_iterator():
    image = i[0]
    label = i[1]
    print(image.shape)
    print(label.decode())

In [ ]:
import csv

with open('coors.csv', mode='r') as infile:
    reader = csv.reader(infile)